### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2025, 1, 23)

In [7]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-01-23
Yesterday: 2025-01-22


In [9]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [11]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [13]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [15]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-22'
ORDER BY a.name



(20, 1)

In [17]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-22' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [20]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-01-22,5.10,5.15,25049174,5.15,5.05,B,SET,0.05,0.98
1,AIMIRT,2025-01-13,2025-01-22,10.50,10.50,11477600,10.60,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-21,2025-01-22,11.90,12.10,4311880,12.20,11.90,B,SET,0.20,1.68
3,DIF,2025-01-16,2025-01-22,8.25,8.45,99319856,8.45,8.20,B,SET,0.20,2.42
4,GVREIT,2025-01-20,2025-01-22,6.65,6.50,7055868,6.65,6.45,B,SET,-0.15,-2.26
5,JMART,2025-01-22,2025-01-22,11.20,11.10,17815986,11.80,10.70,B,SET50,-0.10,-0.89
6,KCE,2025-01-22,2025-01-22,23.20,22.90,26237140,23.20,21.60,B,SET100,-0.30,-1.29
7,MCS,2025-01-22,2025-01-22,6.35,6.25,2104686,6.60,6.25,B,SET,-0.10,-1.57
8,NER,2025-01-17,2025-01-22,4.84,5.00,65405832,5.00,4.82,B,SET,0.16,3.31
9,SENA,2025-01-22,2025-01-22,2.18,2.20,6253440,2.24,2.16,B,SET,0.02,0.92


In [22]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [26]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-22,2025-01-22,5.10,5.15,"25,049,174",5.15,5.05,0.98%,B
1,AIMIRT,2025-01-13,2025-01-22,10.50,10.50,"11,477,600",10.60,10.40,0.00%,B
2,CPNREIT,2025-01-21,2025-01-22,11.90,12.10,"4,311,880",12.20,11.90,1.68%,B
3,DIF,2025-01-16,2025-01-22,8.25,8.45,"99,319,856",8.45,8.20,2.42%,B
4,GVREIT,2025-01-20,2025-01-22,6.65,6.50,"7,055,868",6.65,6.45,-2.26%,B
5,JMART,2025-01-22,2025-01-22,11.20,11.10,"17,815,986",11.80,10.70,-0.89%,B
6,KCE,2025-01-22,2025-01-22,23.20,22.90,"26,237,140",23.20,21.60,-1.29%,B
7,MCS,2025-01-22,2025-01-22,6.35,6.25,"2,104,686",6.60,6.25,-1.57%,B
8,NER,2025-01-17,2025-01-22,4.84,5.00,"65,405,832",5.00,4.82,3.31%,B
9,SENA,2025-01-22,2025-01-22,2.18,2.20,"6,253,440",2.24,2.16,0.92%,B


In [28]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [30]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [32]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [34]:
sales = df[cols]
sales.shape

(20, 10)

In [36]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [38]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [40]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [42]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-22,2025-01-22,5.10,5.15,25049174,5.15,5.05,0.98,B,5.15,0.00,2669457,2025-01-23
1,AIMIRT,2025-01-13,2025-01-22,10.50,10.50,11477600,10.60,10.40,0.00,B,10.40,-0.10,67981,2025-01-23
2,CPNREIT,2025-01-21,2025-01-22,11.90,12.10,4311880,12.20,11.90,1.68,B,12.20,0.10,1813145,2025-01-23
3,DIF,2025-01-16,2025-01-22,8.25,8.45,99319856,8.45,8.20,2.42,B,8.40,-0.05,4605512,2025-01-23
4,GVREIT,2025-01-20,2025-01-22,6.65,6.50,7055868,6.65,6.45,-2.26,B,6.50,0.00,162709,2025-01-23
5,JMART,2025-01-22,2025-01-22,11.20,11.10,17815986,11.80,10.70,-0.89,B,11.20,0.10,2215089,2025-01-23
6,KCE,2025-01-22,2025-01-22,23.20,22.90,26237140,23.20,21.60,-1.29,B,22.70,-0.20,8294906,2025-01-23
7,MCS,2025-01-22,2025-01-22,6.35,6.25,2104686,6.60,6.25,-1.57,B,6.30,0.05,114200,2025-01-23
8,NER,2025-01-17,2025-01-22,4.84,5.00,65405832,5.00,4.82,3.31,B,4.92,-0.08,14525378,2025-01-23
9,SENA,2025-01-22,2025-01-22,2.18,2.20,6253440,2.24,2.16,0.92,B,2.20,0.00,377883,2025-01-23


In [44]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [46]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [48]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [51]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [53]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-22 2025-01-23   5.10   5.15   0.98%  25,049,174   5.15   5.05 B Update
AIMIRT   2025-01-13 2025-01-23  10.50  10.40  -0.95%  11,477,600  10.60  10.40 B Update
AWC      2025-01-23 2025-01-23   3.10   3.08  -0.65% 223,515,896   3.18   3.04 S Insert
BCH      2025-01-23 2025-01-23  15.10  15.00  -0.66%  23,119,392  15.10  14.20 S Insert
CPNREIT  2025-01-21 2025-01-23  11.90  12.20   2.52%   4,311,880  12.20  11.90 B Update
DIF      2025-01-23 2025-01-23   8.45   8.40  -0.59%  99,319,856   8.45   8.20 B Insert
GVREIT   2025-01-20 2025-01-23   6.65   6.50  -2.26%   7,055,868   6.65   6.45 B Update
JMART    2025-01-23 2025-01-23  11.10  11.20    0.9%  17,815,986  11.80  10.70 B Insert
JMT      2025-01-23 2025-01-23  14.80  15.00   1.35%  72,617,738  15.70  14.30 S Insert
KCE      2025-01-22 2025-01-23  

In [55]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [57]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-23' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [59]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [61]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-01-23,5.10,5.15,"50,098,348",5.15,5.05,B,SET,0.05,0.98%
1,AIMIRT,2025-01-13,2025-01-23,10.50,10.40,"22,955,200",10.60,10.40,B,SET,-0.10,-0.95%
2,CPNREIT,2025-01-21,2025-01-23,11.90,12.20,"8,623,760",12.20,11.90,B,SET,0.30,2.52%
3,DIF,2025-01-23,2025-01-23,8.45,8.40,"99,319,856",8.45,8.20,B,SET,-0.05,-0.59%
4,GVREIT,2025-01-20,2025-01-23,6.65,6.50,"14,111,736",6.65,6.45,B,SET,-0.15,-2.26%
5,JMART,2025-01-23,2025-01-23,11.10,11.20,"17,815,986",11.80,10.70,B,SET50,0.10,0.90%
6,KCE,2025-01-22,2025-01-23,23.20,22.70,"52,474,280",23.20,21.60,B,SET100,-0.50,-2.16%
7,MCS,2025-01-23,2025-01-23,6.25,6.30,"2,104,686",6.60,6.25,B,SET,0.05,0.80%
8,NER,2025-01-23,2025-01-23,5.00,4.92,"65,405,832",5.00,4.82,B,SET,-0.08,-1.60%
9,SENA,2025-01-22,2025-01-23,2.18,2.20,"12,506,880",2.24,2.16,B,SET,0.02,0.92%


In [63]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [65]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,TVO,2025-01-02,2025-01-23,22.90,21.00,"62,434,928",23.20,21.00,S,SET,-1.90,-8.30%


In [67]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [69]:
conlite.close()